<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-12 13:44:54
-------------------
qualified stocks: 91
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  3.50 L
Current:  1.35 C
-------------------
Today PnL: -1.11 L (-0.82%)
Current PnL: -15.17 L (-10.57%)
CY Booked + Current PnL: -8.80 L (-6.13%)
-------------------
Total profit:  3.31 L
Total loss:  -18.48 L
-------------------
Total Booked + Current PnL: 18.18 L (15.6%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.73%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 71.07 L (52.72%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 10.27%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,827.06,983.0,-4.34,H-LC,84.62,97603.0,12416.0,3650.0,-1.78,14.57,3.74,18.85,35.0,3.40,0.74,29.95,XY25,NTT,INSURANCE
60,RELIANCE,1291.83,1526.0,5.74,X-LC,30.77,157134.0,16325.0,9192.0,-0.50,11.59,5.85,18.13,17.0,1.78,1.20,24.07,XY25,NTT,REFINERIES
3,ADANIPORTS,1103.69,1583.0,2.51,M-LC,52.75,102801.0,24439.0,9591.0,-0.58,31.19,9.33,43.43,77.0,2.55,0.78,37.50,XY24,NTT,MISC
82,UJJIVANSFB,52.77,53.0,52.86,M-SC,82.42,132813.0,-9666.0,10293.0,0.37,-6.78,7.75,0.44,242.0,-0.94,1.01,56.76,OX40N,NTT,BANKS
83,UNIONBANK,123.87,163.0,-0.31,M-LC,89.01,173756.0,32916.0,11572.0,-0.97,23.37,6.66,31.59,86.0,2.84,1.32,50.93,XY24,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAIL,130.64,228.00,98.52,M-MC,67.03,225771.0,809.0,166845.0,-1.53,0.36,73.90,74.53,184.0,0.00,1.72,31.08,XY24,BTT,STEEL
13,BERGEPAINT,561.33,680.00,-16.65,H-MC,46.15,228420.0,1081.0,46986.0,-0.56,0.48,20.57,21.14,117.0,0.02,1.74,27.89,XY24,NTT,PAINTS
88,VOLTAS,1278.28,1929.20,-29.06,H-MC,40.66,193380.0,1638.0,95994.0,-1.76,0.85,49.64,50.92,99.0,0.02,1.47,7.11,XY25,ATH,AC
56,PGHH,13388.00,18106.42,-20.27,X-MC,7.69,201225.0,405.0,70368.0,0.00,0.20,34.97,35.24,53.0,0.01,1.53,4.89,X40,ATH,FMCG
19,CIPLA,1495.00,1795.00,-17.32,H-LC,24.18,101009.0,844.0,19252.0,-0.93,0.84,19.06,20.07,29.0,0.04,0.77,8.88,X40N,BTT,PHARMA


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TANLA,917.30,1963.11,-27.77,H-SC,98.90,176958.0,-62457.0,335406.0,8.89,-26.09,189.54,114.01,128.0,-0.19,1.35,62.30,AR,ATH,IT
62,ROUTE,1558.40,2249.26,-33.00,M-SC,86.81,72300.0,-30554.0,76154.0,6.67,-29.71,105.33,44.33,188.0,-0.40,0.55,20.06,SR,ATH,IT
85,VALIANTORG,512.64,838.00,-403.18,H-SC,100.00,128934.0,-4352.0,88952.0,2.53,-3.27,68.99,63.47,142.0,-0.05,0.98,116.13,XR,NTT,CHEMICALS
51,LTIM,5564.16,7197.33,1.86,H-LC,71.43,196128.0,-4182.0,62977.0,1.00,-2.09,32.11,29.35,36.0,-0.07,1.49,35.50,X200,ATH,IT
67,SFL,1016.35,1287.00,283.60,H-SC,39.56,173221.0,-88997.0,158826.0,0.87,-33.94,91.69,26.63,145.0,-0.56,1.32,7.70,XY24,NTT,MISC


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-53.72,L-SC,91.21,69235.0,-44314.0,84363.0,-3.40,-39.03,121.85,35.27,260.0,-0.53,0.53,69.45,XR,NTT,HOTELS
27,GREENPANEL,375.16,537.00,198.45,M-SC,87.91,133224.0,-49854.0,128828.0,-2.90,-27.23,96.70,43.14,212.0,-0.39,1.01,25.32,XY24,NTT,MISC
17,CAMS,3643.00,5250.99,2.44,H-SC,74.73,115181.0,13177.0,31848.0,-2.72,12.92,27.65,44.14,125.0,0.41,0.88,31.26,X40N,ATH,MISC
9,AWL,325.39,485.00,-68.50,H-MC,41.76,221595.0,-46852.0,178539.0,-2.68,-17.45,80.57,49.05,98.0,-0.26,1.69,13.93,XY24,NTT,FMCG
77,TATAMOTORS,761.77,1065.00,-52.64,M-LC,29.67,260198.0,-16325.0,126404.0,-2.66,-5.90,48.58,39.81,69.0,-0.13,1.98,24.70,XY24,NTT,AUTO


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,UJJIVANSFB,52.77,53.0,52.86,M-SC,82.42,132813.0,-9666.0,10293.0,0.37,-6.78,7.75,0.44,242.0,-0.94,1.01,56.76,OX40N,NTT,BANKS
74,SYMPHONY,1306.42,1306.0,-27.85,M-SC,3.30,153611.0,-17530.0,17481.0,-1.45,-10.24,11.38,-0.03,190.0,-1.00,1.17,10.40,OX40N,NTT,DURABLES
76,TATAELXSI,7332.28,7332.0,-13.29,X-MC,56.04,91070.0,-11582.0,11575.0,-1.80,-11.28,12.71,-0.00,57.0,-1.00,0.69,38.44,OX40N,NTT,IT
87,VIPIND,488.80,489.0,-949.34,H-SC,92.31,80954.0,-14362.0,14402.0,0.37,-15.07,17.79,0.04,153.0,-1.00,0.62,59.67,OX40N,NTT,MISC
47,KANSAINER,299.63,340.0,-67.67,H-SC,9.89,225360.0,-44307.0,80634.0,-0.16,-16.43,35.78,13.47,144.0,-0.55,1.72,12.80,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-9.91,X-LC,32.97,289282.0,27681.0,117940.0,-0.92,10.58,40.77,55.67,1.0,0.23,2.20,17.33,X40,BTT,IT
78,TCS,3794.03,4998.00,-17.44,X-LC,8.79,268632.0,-27302.0,121207.0,-0.80,-9.23,45.12,31.73,2.0,-0.23,2.05,7.52,X40,BTT,IT
34,HINDUNILVR,2413.81,2723.00,-16.63,X-LC,14.29,261912.0,-8435.0,43058.0,-1.59,-3.12,16.44,12.81,5.0,-0.20,1.99,8.40,XY25,NTT,FMCG
14,BRITANNIA,4983.33,6446.05,12.32,X-LC,25.27,257669.0,28436.0,38856.0,-0.86,12.40,15.08,29.35,7.0,0.73,1.96,22.43,XY25,ATH,FMCG
55,NESTLEIND,2268.60,2755.00,-12.59,X-LC,18.68,277536.0,12110.0,44794.0,-1.95,4.56,16.14,21.44,10.0,0.27,2.11,11.04,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,UNIONBANK,123.87,163.0,-0.31,M-LC,89.01,173756.0,32916.0,11572.0,-0.97,23.37,6.66,31.59,86.0,2.84,1.32,50.93,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.51,M-LC,52.75,102801.0,24439.0,9591.0,-0.58,31.19,9.33,43.43,77.0,2.55,0.78,37.50,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAIL,130.64,228.00,98.52,M-MC,67.03,225771.0,809.0,166845.0,-1.53,0.36,73.90,74.53,184.0,0.00,1.72,31.08,XY24,BTT,STEEL
3,ADANIPORTS,1103.69,1583.00,2.51,M-LC,52.75,102801.0,24439.0,9591.0,-0.58,31.19,9.33,43.43,77.0,2.55,0.78,37.50,XY24,NTT,MISC
17,CAMS,3643.00,5250.99,2.44,H-SC,74.73,115181.0,13177.0,31848.0,-2.72,12.92,27.65,44.14,125.0,0.41,0.88,31.26,X40N,ATH,MISC
32,HEROMOTOCO,4311.81,6039.03,0.48,H-MC,63.74,153237.0,2324.0,58123.0,-0.73,1.54,37.93,40.06,100.0,0.04,1.17,25.34,AR,ATH,AUTO
83,UNIONBANK,123.87,163.00,-0.31,M-LC,89.01,173756.0,32916.0,11572.0,-0.97,23.37,6.66,31.59,86.0,2.84,1.32,50.93,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,WHIRLPOOL,1167.49,2270.00,-31.38,M-SC,53.85,199920.0,28299.0,133766.0,0.78,16.49,66.91,94.43,194.0,0.21,1.52,47.55,XR,NTT,DURABLES
11,BANKINDIA,116.91,190.00,-28.22,M-MC,75.82,190635.0,10827.0,101589.0,-1.85,6.02,53.29,62.52,170.0,0.11,1.45,37.19,XR,NTT,BANKS
90,WIPRO,243.46,420.00,-6.08,M-LC,37.36,161231.0,10286.0,99173.0,0.44,6.81,61.51,72.51,68.0,0.10,1.23,11.23,XR,NTT,IT
39,INDIAMART,2327.09,4871.06,-49.23,H-SC,64.84,132182.0,8846.0,125983.0,-0.16,7.17,95.31,109.32,122.0,0.07,1.01,30.59,AR,ATH,MISC
32,HEROMOTOCO,4311.81,6039.03,0.48,H-MC,63.74,153237.0,2324.0,58123.0,-0.73,1.54,37.93,40.06,100.0,0.04,1.17,25.34,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-53.72,L-SC,91.21,69235.0,-44314.0,84363.0,-3.40,-39.03,121.85,35.27,260.0,-0.53,0.53,69.45,XR,NTT,HOTELS
52,MASFIN,326.60,399.50,-21.86,H-SC,65.93,88965.0,-9015.0,30889.0,-1.61,-9.20,34.72,22.32,141.0,-0.29,0.68,29.03,XR,ATH,FINANCE
50,LICI,827.06,983.00,-4.34,H-LC,84.62,97603.0,12416.0,3650.0,-1.78,14.57,3.74,18.85,35.0,3.40,0.74,29.95,XY25,NTT,INSURANCE
3,ADANIPORTS,1103.69,1583.00,2.51,M-LC,52.75,102801.0,24439.0,9591.0,-0.58,31.19,9.33,43.43,77.0,2.55,0.78,37.50,XY24,NTT,MISC
35,HINDZINC,514.80,744.74,37.93,H-LC,90.11,103290.0,330.0,45654.0,-0.79,0.32,44.20,44.67,25.0,0.01,0.79,33.29,X5K,ATH,METALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VALIANTORG,512.64,838.00,-403.18,H-SC,100.0,128934.0,-4352.0,88952.0,2.53,-3.27,68.99,63.47,142.0,-0.05,0.98,116.13,XR,NTT,CHEMICALS
75,TANLA,917.30,1963.11,-27.77,H-SC,98.9,176958.0,-62457.0,335406.0,8.89,-26.09,189.54,114.01,128.0,-0.19,1.35,62.30,AR,ATH,IT
7,ASIANTILES,75.41,137.00,6783.33,M-SC,97.8,75536.0,-18274.0,94896.0,-2.27,-19.48,125.63,81.67,238.0,-0.19,0.58,46.28,XR,NTT,CERAMICS
46,JPPOWER,18.73,26.20,-33.51,L-SC,96.7,135557.0,-6285.0,62858.0,-1.76,-4.43,46.37,39.88,258.0,-0.10,1.03,37.38,XY24,NTT,POWER
79,TITAGARH,1097.23,1548.00,-8.60,H-SC,95.6,160685.0,-31330.0,110214.0,-1.94,-16.32,68.59,41.08,151.0,-0.28,1.22,36.00,XY24,NTT,ENGINEERING


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.66
1,25,43.14
2,50,72.00


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.98
XY25     14.23
X40      12.53
XR       11.06
OX40N     8.54
X40N      8.50
AR        8.12
X5K       2.34
X200      1.49
SR        1.21
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.87
X-LC    15.52
M-SC    14.38
H-LC    11.07
H-MC     8.96
M-LC     8.52
X-MC     7.69
M-MC     5.81
X-SC     2.82
L-SC     2.50
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.60,-2.09,36.25
IT,11.40,-10.91,67.21
BANKS,8.59,-7.72,51.13
MISC,6.62,-24.59,73.31
FINANCE,6.42,-19.55,65.55
PAINTS,5.68,-17.28,40.39
ELECTRICAL,5.16,-3.68,48.66
HEALTHCARE,4.77,-2.95,33.02
AUTO,4.69,-9.71,57.95


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2731304.0,25
XR,1108332.0,13
AR,1004587.0,9
X40,675363.0,9
XY25,517645.0,10
X40N,415001.0,8
OX40N,309951.0,11
SR,186473.0,2
X5K,95185.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1890749.0,19
M-SC,1437714.0,18
X-LC,625746.0,9
M-MC,580416.0,5
H-MC,551288.0,7
H-LC,501381.0,10
X-MC,407493.0,6
M-LC,324659.0,6
L-SC,269519.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,895721.0,6
M-SC,XY24,575399.0,5
H-SC,AR,515446.0,3
M-MC,XY24,392116.0,3
M-SC,XR,343732.0,4
X-LC,X40,316675.0,3
H-MC,XY24,225525.0,2
M-LC,XY24,210146.0,4
H-SC,XR,192050.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
